<a href="https://colab.research.google.com/github/Md-Thahir/AI-Tools-Lab/blob/main/Exp_13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
data=pd.read_csv("/content/drive/MyDrive/AI Tools Lab/california_housing_2.csv")
data.isnull().mean() * 100

longitude                      0.000000
latitude                       0.000000
housing_median_age             0.000000
total_rooms                    0.000000
total_bedrooms                 1.002907
population                     0.000000
households                     0.000000
median_income                  0.000000
median_house_value             0.000000
ocean_proximity                0.000000
diff_income_and_house_value    0.000000
dtype: float64

In [ ]:
# handling null values in total_bedrooms with the most frequent value in respective column
data["total_bedrooms"].fillna(data['total_bedrooms'].mode()[0],inplace=True)
#checking the null values handled or not
data["total_bedrooms"].isnull().mean() * 100

0.0

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   longitude                    20640 non-null  float64
 1   latitude                     20640 non-null  float64
 2   housing_median_age           20640 non-null  float64
 3   total_rooms                  20640 non-null  float64
 4   total_bedrooms               20640 non-null  float64
 5   population                   20640 non-null  float64
 6   households                   20640 non-null  float64
 7   median_income                20640 non-null  float64
 8   median_house_value           20640 non-null  float64
 9   ocean_proximity              20640 non-null  object 
 10  diff_income_and_house_value  20640 non-null  float64
dtypes: float64(10), object(1)
memory usage: 1.7+ MB


In [ ]:
print(data['ocean_proximity'].unique())

['<1H OCEAN' 'INLAND' 'NEAR OCEAN' 'NEAR BAY' 'ISLAND']


In [ ]:
print(data['ocean_proximity'].value_counts())

ocean_proximity
<1H OCEAN     9136
INLAND        6551
NEAR OCEAN    2658
NEAR BAY      2290
ISLAND           5
Name: count, dtype: int64


In [ ]:
#we need to convert categorical values by label encoding
# there are more than two categories, we have to use onehot encoding
ocean_prox_df = pd.get_dummies(data['ocean_proximity'],drop_first=True,dtype=int)
ocean_prox_df.head()

,INLAND,ISLAND,NEAR BAY,NEAR OCEAN
0,0,0,0,0
1,1,0,0,0
2,0,0,0,1
3,0,0,1,0
4,0,0,1,0


In [ ]:
old_data = data.copy()
data.drop(['ocean_proximity','longitude','latitude','diff_income_and_house_value'],axis=1,inplace=True)
data.head()

,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,29.0,515.0,229.0,2690.0,217.0,0.4999,500001.0
1,28.0,238.0,58.0,142.0,31.0,0.4999,500001.0
2,19.0,1540.0,715.0,1799.0,635.0,0.7025,500001.0
3,27.0,1728.0,884.0,1211.0,752.0,0.8543,500001.0
4,52.0,3260.0,1535.0,3260.0,1457.0,0.9000,500001.0


In [ ]:
data = pd.concat([data,ocean_prox_df],axis=1)
data.head()

,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,INLAND,ISLAND,NEAR BAY,NEAR OCEAN
0,29.0,515.0,229.0,2690.0,217.0,0.4999,500001.0,0,0,0,0
1,28.0,238.0,58.0,142.0,31.0,0.4999,500001.0,1,0,0,0
2,19.0,1540.0,715.0,1799.0,635.0,0.7025,500001.0,0,0,0,1
3,27.0,1728.0,884.0,1211.0,752.0,0.8543,500001.0,0,0,1,0
4,52.0,3260.0,1535.0,3260.0,1457.0,0.9000,500001.0,0,0,1,0


In [ ]:
from sklearn.model_selection import train_test_split
# split the data for training and testing
X_train, X_test, y_train, y_test = train_test_split(data.drop('median_house_value',axis=1),
data['median_house_value'], test_size=0.30,
random_state=101)

A)    Linear Regression with Polynomial Features

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
#model initialization
model = LinearRegression()
# initializing polynomial featuers
poly = PolynomialFeatures(degree=3)
#converting features into polyfeatures
X_ = poly.fit_transform(X_train)
Y_ = poly.fit_transform(y_train.values.reshape(-1,1))
# training the model
model.fit(X_,Y_)

LinearRegression()

In [ ]:
#preparing test data for predictions
testX = poly.fit_transform(X_test)
# predicting the output for test data
predicted = model.predict(testX)
# expected output for test data
expected = poly.fit_transform(y_test.values.reshape(-1,1))
from sklearn.metrics import r2_score
r2 = r2_score(expected, predicted)
print('r2 score is', r2)

r2 score is 0.5907072614542763


In [ ]:
# example of calculate the root mean squared error
from sklearn.metrics import mean_squared_error
# calculate errors
errors = mean_squared_error(expected, predicted, squared=False)
# report error
print("root mean square error is :",errors)

root mean square error is : 5960662955758872.0



B)    Random Forest Regressor



In [ ]:
# Fitting Random Forest Regression to the dataset
# import the regressor
from sklearn.ensemble import RandomForestRegressor
# create regressor object
regressor = RandomForestRegressor(n_estimators = 100, random_state = 101)
# fit the regressor with x and y data
regressor.fit(X_train, y_train)

RandomForestRegressor(random_state=101)

In [ ]:
# test the output by changing values
predicted = regressor.predict(X_test)
expected = y_test
from sklearn.metrics import r2_score
r2 = r2_score(expected, predicted)
print('r2 score is', r2)

r2 score is 0.7091234171276952


In [ ]:
# example of calculate the root mean squared error
from sklearn.metrics import mean_squared_error
# calculate errors
errors =mean_squared_error(expected, predicted,squared=False)
# report error
print("root mean square error is :",errors)

root mean square error is : 62360.02542136252
